# Tests for Face detection with OpenCV
the following notebook contains some test with different model/images for face detection. 

In [2]:
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
print("OpenCV: "+ str(cv2.__version__))
print("IMPORTS OK")

OpenCV: 4.5.3
IMPORTS OK


In [3]:
image_path = "./img/faces.png"
show_image = False #show preview image in notebook
WIDTH_ROBOT = 960
HEIGHT_ROBOT = 600

In [4]:
#Read image
img_original = cv2.imread(image_path,cv2.IMREAD_COLOR )
#calculate h and w
(h, w) = img_original.shape[:2]
WIDTH = w
HEIGHT = h
print("dimensions image upload: "+ str((h,w)) )
print("dimensions robot camera: "+ str((HEIGHT_ROBOT, WIDTH_ROBOT)) )
print("Same dimensions: "+str((h,w)==(HEIGHT, WIDTH)))

def imshow(image_passed, save=False):
    if show_image:
        #cv2.imshow("image",image_passed) #not working in Jupiter notebook
        plt.imshow(cv2.cvtColor(image_passed, cv2.COLOR_BGR2RGB))
        plt.show()
    if save:
        #SAVE
        cv2.imwrite("./img/save.png",image_passed)
        print("Saved")

#SHOW IMAGE        
imshow(img_original)

dimensions image upload: (600, 960)
dimensions robot camera: (600, 960)
Same dimensions: True


## HAAR CLASSIFIER FACE DETECTION

In [5]:
#Load Classifier
CASC_PATH = "./models/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(CASC_PATH)
print("Classifier Loaded")

Classifier Loaded


In [6]:
img_haar = img_original.copy() #copy img

#SHOW IMAGE        
imshow(img_haar)

#FORWARD
faces = face_cascade.detectMultiScale(img_haar, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#print("faces detected: ",faces.shape[0])
print("Detected: "+str(len(faces)))

#draw bounding boxes
for face in faces:
    x, y, w, h = face
    cv2.rectangle(img_haar, (x, y), (x+w, y+h), (0, 255, 0), 2)


#SHOW IMAGE        
imshow(img_haar, save=True)

Detected: 15
Saved


## DLIB (HOG)

In [7]:
import cv2
import dlib  #requires pip install dlib
#Load Classifier
detector = dlib.get_frontal_face_detector()
print("Classifier Loaded")

Classifier Loaded


In [8]:
img_dlib = img_original.copy() #copy img

#SHOW IMAGE        
imshow(img_dlib)

#FORWARD
gray = cv2.cvtColor(img_dlib, cv2.COLOR_BGR2GRAY)
faces = detector(gray, 1) # result
print("Detected: "+str(len(faces)))

#to draw faces on image
for result in faces:
    x1 = result.left()
    y1 = result.top()
    x2 = result.right()
    y2 = result.bottom()
    cv2.rectangle(img_dlib, (x1, y1), (x2, y2), (0,255 , 0), 2)
    

#SHOW IMAGE        
imshow(img_dlib, save=True)

Detected: 18
Saved


## DNN FACE DETECTION

In [9]:
#Load net
PROTO = "./models/res10_deploy.prototxt"
MODEL = "./models/res10_300x300_ssd_iter_140000_fp16.caffemodel"
net = cv2.dnn.readNetFromCaffe(PROTO, MODEL)
print("Net Loaded")

Net Loaded


In [10]:
img_dnn = img_original.copy() #copy img

#SHOW IMAGE        
imshow(img_dnn)

#FORWARD image
#blob = cv2.dnn.blobFromImage(cv2.resize(img_dnn, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
blob = cv2.dnn.blobFromImage(img_dnn, 1.0, (300, 300), [104, 117, 123], False, False)
net.setInput(blob)
detections = net.forward()
print("Detected: " + str(detections.shape[2]))
detected_after_threshold=0

#draw bounding box
conf_threshold=0.15
bboxes = []
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_threshold:
        detected_after_threshold+=1
        x1 = int(detections[0, 0, i, 3] * WIDTH)
        y1 = int(detections[0, 0, i, 4] * HEIGHT)
        x2 = int(detections[0, 0, i, 5] * WIDTH)
        y2 = int(detections[0, 0, i, 6] * HEIGHT)
        cv2.rectangle(img_dnn, (x1, y1), (x2, y2), (0, 255, 0), 2)
print("Detected: " + str(detected_after_threshold) + " (after threshold)")


#SHOW IMAGE        
imshow(img_dnn, save=True)

Detected: 200
Detected: 14 (after threshold)
Saved


## OpenCV FaceDetector

In [11]:
#Load net
CONFIG = "./models/opencv_face_detector.pbtxt"
MODEL = "./models/opencv_face_detector_uint8.pb"
net = cv2.dnn.readNetFromTensorflow(MODEL, CONFIG)
print("Net Loaded")

Net Loaded


In [12]:
img_cvdet = img_original.copy() #copy img

#SHOW IMAGE        
imshow(img_cvdet)

#FORWARD image
blob = cv2.dnn.blobFromImage(img_cvdet, 1.0, (300, 300), [104, 117, 123], True, False)
net.setInput(blob)
detections = net.forward()
print("Detected: " + str(detections.shape[2]))
detected_after_threshold=0

#draw bounding box
conf_threshold=0.15
bboxes = []
for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > conf_threshold:
        detected_after_threshold+=1
        x1 = int(detections[0, 0, i, 3] * WIDTH)
        y1 = int(detections[0, 0, i, 4] * HEIGHT)
        x2 = int(detections[0, 0, i, 5] * WIDTH)
        y2 = int(detections[0, 0, i, 6] * HEIGHT)
        bboxes.append([x1, y1, x2, y2])
        cv2.rectangle(img_cvdet, (x1, y1), (x2, y2), (0, 255, 0), 2)
print("Detected: " + str(detected_after_threshold) + " (after threshold)")


#SHOW IMAGE        
imshow(img_cvdet, save=True)

Detected: 200
Detected: 1 (after threshold)
Saved
